# **Benchmarking Multi-Agent Coordination for Energy Planning**

In [ ]:
!pip -q install openai numpy pandas scikit-learn

import os
import json
import re
import time
import random
import numpy as np
import pandas as pd
from dataclasses import dataclass, asdict
from typing import List, Dict, Any
from sklearn.ensemble import RandomForestRegressor
from openai import OpenAI

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

os.environ["GROQ_API_KEY"] = "PUT_YOUR_KEY_HERE"

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"]
)
LLM_MODEL = "llama-3.1-8b-instant"

LLM_TEMPERATURE = 0.25
LLM_MAX_TOKENS = 350
LLM_RETRIES = 2
HARD_SECURITY = 0.90

url = "https://github.com/owid/energy-data/raw/master/owid-energy-data.csv"
try:
    df = pd.read_csv(url)
except:
    if os.path.exists("/content/owid-energy-data.csv"):
        df = pd.read_csv("/content/owid-energy-data.csv")
    else:
        data = {
            'iso_code': ['AUT']*30 + ['BEL']*30,
            'country': ['Austria']*30 + ['Belgium']*30,
            'year': list(range(2000, 2030))*2,
            'electricity_demand': np.random.uniform(50, 100, 60),
            'carbon_intensity_elec': np.random.uniform(200, 400, 60),
            'low_carbon_share_elec': np.random.uniform(10, 50, 60),
            'fossil_share_elec': np.random.uniform(10, 50, 60),
            'net_elec_imports_share_demand': np.random.uniform(-10, 10, 60)
        }
        df = pd.DataFrame(data)

df = df[~df["iso_code"].astype(str).str.startswith("OWID_")].copy()
df["year"] = df["year"].astype(int)

@dataclass
class Episode:
    episode_id: str
    country: str
    iso_code: str
    train_years: np.ndarray
    test_years: np.ndarray
    demand_train: np.ndarray
    demand_test: np.ndarray
    intensity_train: np.ndarray
    intensity_test: np.ndarray
    base_low_carbon_pct: float
    base_fossil_pct: float
    base_net_import_pct: float
    w_cost: float
    w_emission: float
    w_security: float
    budget_cap: float
    max_emission: float
    min_security: float

@dataclass
class Plan:
    renewable_invest: float
    storage_invest: float
    demand_response: float
    carbon_tax: float
    def clip(self):
        self.renewable_invest=float(np.clip(self.renewable_invest,0,1))
        self.storage_invest=float(np.clip(self.storage_invest,0,1))
        self.demand_response=float(np.clip(self.demand_response,0,1))
        self.carbon_tax=float(np.clip(self.carbon_tax,0,1))
        return self

PLAN_SCHEMA = ["renewable_invest", "storage_invest", "demand_response", "carbon_tax"]

def build_episodes(df, n=30):
    episodes = []
    valid_iso = df.groupby("iso_code").filter(lambda x: x["year"].max() >= 2022 and len(x) > 25)["iso_code"].unique()

    candidates = []
    for iso in valid_iso:
        sub = df[df.iso_code == iso]
        count = sub["electricity_demand"].notna().sum()
        candidates.append((iso, count))

    candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:n]
    top_isos = [x[0] for x in candidates]

    for iso in top_isos:
        sub = df[df.iso_code == iso].sort_values("year").copy()
        sub["electricity_demand"] = sub["electricity_demand"].ffill().fillna(0)
        sub["carbon_intensity_elec"] = sub["carbon_intensity_elec"].ffill().fillna(400)

        train = sub[sub.year.between(2000, 2018)]
        test = sub[sub.year.between(2019, 2023)]

        if len(train) < 10 or len(test) < 2:
            continue

        last = train.iloc[-1]

        episodes.append(Episode(
            episode_id=f"{iso}_Medium",
            country=sub.iloc[0]["country"],
            iso_code=iso,
            train_years=train.year.values,
            test_years=test.year.values,
            demand_train=train.electricity_demand.values,
            demand_test=test.electricity_demand.values,
            intensity_train=train.carbon_intensity_elec.values,
            intensity_test=test.carbon_intensity_elec.values,
            base_low_carbon_pct=float(last.get("low_carbon_share_elec", 10)),
            base_fossil_pct=float(last.get("fossil_share_elec", 80)),
            base_net_import_pct=float(last.get("net_elec_imports_share_demand", 0)),
            w_cost=0.4, w_emission=0.4, w_security=0.2,
            budget_cap=0.60,
            max_emission=0.45,
            min_security=0.90
        ))
    return episodes

def evaluate_plan(ep: Episode, plan: Plan, demand_forecast: np.ndarray) -> Dict[str, float]:
    MAX_DEM = np.max(ep.demand_train) * 1.5 if len(ep.demand_train)>0 else 100
    avg_dem = np.mean(demand_forecast) * (1 - 0.2*plan.demand_response)

    base_int = np.mean(ep.intensity_train) if len(ep.intensity_train) else 400
    red = (plan.renewable_invest * 0.6) + (plan.carbon_tax * 0.2)
    curr_int = base_int * (1 - red)
    emission = np.clip((curr_int / 800) * (avg_dem/MAX_DEM), 0, 1)

    base_cost = 0.3*plan.renewable_invest + 0.4*plan.storage_invest + 0.1*plan.carbon_tax
    grid_penalty = 0.0
    if plan.renewable_invest > (plan.storage_invest + 0.1):
        grid_penalty = 0.15
    cost = np.clip(base_cost + grid_penalty, 0, 1)

    sec_base = 0.85
    storage_bonus = 0.55 * plan.storage_invest
    renewable_penalty = 0.10 * plan.renewable_invest
    sec = np.clip(sec_base + storage_bonus - renewable_penalty, 0, 1)

    budget_over = max(0.0, cost - ep.budget_cap) / max(1e-6, (1.0 - ep.budget_cap))
    emission_over = max(0.0, emission - ep.max_emission) / max(1e-6, (1.0 - ep.max_emission))
    security_under = max(0.0, ep.min_security - sec) / max(1e-6, ep.min_security)

    total_violation = float(budget_over + emission_over + security_under)
    any_violation = float(total_violation > 1e-9)

    return {
        "cost": float(cost),
        "emission": float(emission),
        "security": float(sec),
        "violations": float(total_violation),
        "any_violation": float(any_violation),
    }

def score(ep: Episode, met: dict) -> float:
    base_score = ep.w_cost*(1-met["cost"]) + ep.w_emission*(1-met["emission"]) + ep.w_security*(met["security"])
    k = 0.9
    penalty = float(np.exp(-k * met["violations"]))
    return float(base_score * penalty)

def forecast_ml(ep: Episode):
    X_train, y_train = [], []
    for i in range(1, len(ep.train_years)):
        prev_demand = ep.demand_train[i-1]
        year = ep.train_years[i]
        target = ep.demand_train[i]
        X_train.append([year, prev_demand])
        y_train.append(target)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    if len(y_train) < 5:
        yhat = np.full_like(ep.test_years, np.mean(ep.demand_train), dtype=float)
        denom = np.clip(ep.demand_test, 1.0, None)
        mape = np.mean(np.abs((ep.demand_test - yhat)/denom)) * 100
        return yhat, float(mape)

    model = RandomForestRegressor(n_estimators=150, max_depth=8, random_state=SEED)
    model.fit(X_train, y_train)

    preds = []
    last_demand = float(ep.demand_train[-1])
    for year in ep.test_years:
        feat = np.array([[year, last_demand]])
        pred = float(model.predict(feat)[0])
        preds.append(pred)
        last_demand = pred

    yhat = np.array(preds, dtype=float)
    denom = np.clip(ep.demand_test, 1.0, None)
    mape = float(np.mean(np.abs((ep.demand_test - yhat)/denom)) * 100)
    return yhat, mape

def llm_call(prompt, system="You are an AI assistant."):
    for _ in range(LLM_RETRIES + 1):
        try:
            resp = client.chat.completions.create(
                model=LLM_MODEL,
                messages=[{"role":"system","content":system},{"role":"user","content":prompt}],
                temperature=LLM_TEMPERATURE,
                max_tokens=LLM_MAX_TOKENS
            )
            return resp.choices[0].message.content
        except Exception:
            time.sleep(1.0)
    return "{}"

def extract_json(text: str) -> Dict[str, Any]:
    if not isinstance(text, str):
        return {}
    text = text.replace("```json", "").replace("```", "").strip()
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        return {}
    candidate = text[start:end+1]
    try:
        return json.loads(candidate)
    except:
        return {}

def get_ai_proposal(agent_name, goal, plan, metrics):
    sys = (
        f"You are {agent_name}. Goal: {goal}.\n"
        "Return ONLY JSON in this exact format:\n"
        '{"candidates":[{"delta":{"renewable_invest":0.0,"storage_invest":0.0,"demand_response":0.0,"carbon_tax":0.0}},'
        '{"delta":{"renewable_invest":0.0,"storage_invest":0.0,"demand_response":0.0,"carbon_tax":0.0}}]}\n'
        "Rules:\n"
        "- Each delta value must be in [-0.15, +0.15]\n"
        "- In EACH candidate, at least ONE value must be NON-ZERO (e.g., 0.03)\n"
        "- Keep changes small and safe.\n"
    )
    prompt = f"Current plan: {asdict(plan)}\nCurrent metrics: {metrics}\nPropose 2 candidate deltas."
    res = llm_call(prompt, sys)
    d = extract_json(res)

    cands = d.get("candidates", [])
    out = []
    if isinstance(cands, list):
        for c in cands[:2]:
            dd = c.get("delta", {}) if isinstance(c, dict) else {}
            clean = {}
            if isinstance(dd, dict):
                for k, v in dd.items():
                    if k in PLAN_SCHEMA:
                        try:
                            clean[k] = float(np.clip(float(v), -0.15, 0.15))
                        except:
                            pass
            out.append(clean)

    if not out:
        out = [{"storage_invest": 0.03}, {"renewable_invest": 0.03}]
    return out

def apply_best_delta(ep: Episode, plan: Plan, yhat: np.ndarray, delta_list: List[Dict[str, float]]) -> Plan:
    base_met = evaluate_plan(ep, plan, yhat)
    best_s = score(ep, base_met)
    best_plan = plan

    for d in (delta_list or []):
        p_d = asdict(plan)
        for k, v in d.items():
            if k in p_d:
                p_d[k] += float(v)
        cand_plan = Plan(**p_d).clip()
        cand_met = evaluate_plan(ep, cand_plan, yhat)
        cand_s = score(ep, cand_met)
        if cand_s > best_s:
            best_s = cand_s
            best_plan = cand_plan

    return best_plan

def get_initial_plan():
    return Plan(0.20, 0.15, 0.05, 0.05).clip()

def run_baseline_monolith(ep: Episode):
    t0 = time.time()
    yhat, mape = forecast_ml(ep)
    plan = Plan(0.22, 0.16, 0.06, 0.06).clip()
    met = evaluate_plan(ep, plan, yhat)
    return {"method":"baseline_monolith","score":score(ep,met),"forecast_mape":mape,
            **{k:met[k] for k in ["cost","emission","security","violations"]},
            "messages":0,"rounds":1,"vetos":0,"runtime_sec":time.time()-t0}

def run_single_llm(ep: Episode):
    t0 = time.time()
    yhat, mape = forecast_ml(ep)
    sys = (
        "You are an Energy Planner.\n"
        "Return ONLY JSON with fields:\n"
        '{"renewable_invest":0.0,"storage_invest":0.0,"demand_response":0.0,"carbon_tax":0.0}\n'
        "Values 0..1.\n"
        f"Constraints: budget_cost <= {ep.budget_cap}, emission <= {ep.max_emission}, security >= {ep.min_security}.\n"
    )
    prompt = f"Country: {ep.country}\nSuggest a plan.\nCurrent suggestion: {asdict(get_initial_plan())}\n"
    d = extract_json(llm_call(prompt, sys))
    plan = Plan(d.get("renewable_invest",0.22), d.get("storage_invest",0.16),
                d.get("demand_response",0.06), d.get("carbon_tax",0.06)).clip()
    met = evaluate_plan(ep, plan, yhat)
    return {"method":"single_llm_orchestrator","score":score(ep,met),"forecast_mape":mape,
            **{k:met[k] for k in ["cost","emission","security","violations"]},
            "messages":1,"rounds":1,"vetos":0,"runtime_sec":time.time()-t0}

def run_multi_agent_simple(ep: Episode, rounds=3):
    t0 = time.time()
    yhat, mape = forecast_ml(ep)
    plan = get_initial_plan()
    msgs = 0

    for r in range(rounds):
        met = evaluate_plan(ep, plan, yhat)
        cand1 = get_ai_proposal("CostAgent", "Minimize cost while respecting constraints", plan, met)
        plan = apply_best_delta(ep, plan, yhat, cand1); msgs += 1

        met = evaluate_plan(ep, plan, yhat)
        cand2 = get_ai_proposal("EmissionAgent", "Minimize emission while respecting constraints", plan, met)
        plan = apply_best_delta(ep, plan, yhat, cand2); msgs += 1

        met = evaluate_plan(ep, plan, yhat)
        cand3 = get_ai_proposal("SecurityAgent", "Maximize security while respecting constraints", plan, met)
        plan = apply_best_delta(ep, plan, yhat, cand3); msgs += 1

    met = evaluate_plan(ep, plan, yhat)
    return {"method":"multi_agent_simple_agg","score":score(ep,met),"forecast_mape":mape,
            **{k:met[k] for k in ["cost","emission","security","violations"]},
            "messages":msgs,"rounds":rounds,"vetos":0,"runtime_sec":time.time()-t0}

def run_multi_agent_llm_coord(ep: Episode, rounds=3):
    t0 = time.time()
    yhat, mape = forecast_ml(ep)
    plan = get_initial_plan()
    msgs = 0
    vetos = 0

    for r in range(rounds):
        met = evaluate_plan(ep, plan, yhat)

        prop_cost = get_ai_proposal("CostAgent", "Minimize cost while respecting constraints", plan, met)
        prop_emit = get_ai_proposal("EmissionAgent", "Minimize emission while respecting constraints", plan, met)
        prop_sec  = get_ai_proposal("SecurityAgent", "Maximize security while respecting constraints", plan, met)
        msgs += 3

        is_soft_veto = met["security"] < ep.min_security
        is_hard_veto = met["security"] < HARD_SECURITY
        p_d = asdict(plan)

        if is_hard_veto:
            vetos += 1
            deficit = (ep.min_security + 0.01) - met["security"]
            needed_storage = max(0.0, deficit / 0.55)
            p_d["storage_invest"] = float(np.clip(p_d["storage_invest"] + needed_storage, 0, 1))
            p_d["renewable_invest"] = float(np.clip(p_d["renewable_invest"] - 0.05 * needed_storage, 0, 1))
            plan = Plan(**p_d).clip()
            msgs += 1
        else:
            if is_soft_veto:
                merged = (prop_cost or []) + (prop_emit or []) + (prop_sec or [])
                plan = apply_best_delta(ep, plan, yhat, merged)
                msgs += 1
                continue

            sys = (
                "You are a Coordinator.\n"
                "Return ONLY JSON:\n"
                '{"final_delta":{"renewable_invest":0.0,"storage_invest":0.0,"demand_response":0.0,"carbon_tax":0.0}}\n'
                "Each delta in [-0.15,+0.15].\n"
            )
            proposals = {"CostAgent":prop_cost,"EmissionAgent":prop_emit,"SecurityAgent":prop_sec}
            prompt = f"Plan: {p_d}\nMetrics: {met}\nAgent candidates: {proposals}\nReturn final_delta."
            d = extract_json(llm_call(prompt, sys)).get("final_delta", {})
            msgs += 1

            if isinstance(d, dict) and len(d) > 0:
                for k, v in d.items():
                    if k in p_d:
                        try:
                            p_d[k] += float(np.clip(float(v), -0.15, 0.15))
                        except:
                            pass
                plan = Plan(**p_d).clip()
            else:
                merged = (prop_cost or []) + (prop_emit or []) + (prop_sec or [])
                plan = apply_best_delta(ep, plan, yhat, merged)

        met2 = evaluate_plan(ep, plan, yhat)
        if met2["security"] < HARD_SECURITY:
            vetos += 1
            deficit = (ep.min_security + 0.005) - met2["security"]
            needed_storage = max(0.0, deficit / 0.55)
            p_d2 = asdict(plan)
            p_d2["storage_invest"] = float(np.clip(p_d2["storage_invest"] + needed_storage, 0, 1))
            plan = Plan(**p_d2).clip()
            msgs += 1

    met = evaluate_plan(ep, plan, yhat)
    return {"method":"multi_agent_llm_coord","score":score(ep,met),"forecast_mape":mape,
            **{k:met[k] for k in ["cost","emission","security","violations"]},
            "messages":msgs,"rounds":rounds,"vetos":vetos,"runtime_sec":time.time()-t0}

def run_ablation_no_veto(ep: Episode, rounds=3):
    t0 = time.time()
    yhat, mape = forecast_ml(ep)
    plan = get_initial_plan()
    msgs = 0
    vetos = 0

    for r in range(rounds):
        met = evaluate_plan(ep, plan, yhat)

        d_cost = get_ai_proposal("CostAgent", "Minimize cost while respecting constraints", plan, met)
        d_emit = get_ai_proposal("EmissionAgent", "Minimize emission while respecting constraints", plan, met)
        d_sec  = get_ai_proposal("SecurityAgent", "Maximize security while respecting constraints", plan, met)
        msgs += 3

        sys = (
            "You are a Coordinator (NO VETO ABLATION).\n"
            "Return ONLY JSON:\n"
            '{"final_delta":{"renewable_invest":0.0,"storage_invest":0.0,"demand_response":0.0,"carbon_tax":0.0}}\n'
        )
        prompt = f"Plan: {asdict(plan)}\nMetrics: {met}\nAgent candidates: {{'Cost':{d_cost},'Emission':{d_emit},'Security':{d_sec}}}\nReturn final_delta."
        d_final = extract_json(llm_call(prompt, sys)).get("final_delta", {})
        msgs += 1

        if isinstance(d_final, dict) and len(d_final) > 0:
            p_d = asdict(plan)
            for k, v in d_final.items():
                if k in p_d:
                    try:
                        p_d[k] += float(np.clip(float(v), -0.15, 0.15))
                    except:
                        pass
            plan = Plan(**p_d).clip()
        else:
            merged = (d_cost or []) + (d_emit or []) + (d_sec or [])
            plan = apply_best_delta(ep, plan, yhat, merged)

        met2 = evaluate_plan(ep, plan, yhat)
        if met2["security"] < HARD_SECURITY:
            vetos += 1

    met = evaluate_plan(ep, plan, yhat)
    return {"method":"ablation_no_veto","score":score(ep,met),"forecast_mape":mape,
            **{k:met[k] for k in ["cost","emission","security","violations"]},
            "messages":msgs,"rounds":rounds,"vetos":vetos,"runtime_sec":time.time()-t0}

def run_ablation_no_comm(ep: Episode):
    t0 = time.time()
    yhat, mape = forecast_ml(ep)
    plan = get_initial_plan()
    met = evaluate_plan(ep, plan, yhat)

    cand = get_ai_proposal("CostAgent", "Minimize cost while respecting constraints", plan, met)
    plan = apply_best_delta(ep, plan, yhat, cand)

    met = evaluate_plan(ep, plan, yhat)
    return {"method":"ablation_no_comm","score":score(ep,met),"forecast_mape":mape,
            **{k:met[k] for k in ["cost","emission","security","violations"]},
            "messages":1,"rounds":1,"vetos":0,"runtime_sec":time.time()-t0}

try:
    episodes = build_episodes(df, n=15)
    print(f"Seçilen Ülke Sayısı: {len(episodes)}")
except Exception as e:
    print(f"Bölüm oluşturma hatası: {e}")
    episodes = []

results = []

if len(episodes) > 0:
    print("Simülasyon başlıyor (FAIR + High Score + Soft Constraints + Candidate Search)...")
    for i, ep in enumerate(episodes):
        print(f"[{i+1}/{len(episodes)}] İşleniyor: {ep.country} ...")
        results.append({**run_baseline_monolith(ep), "country": ep.country})
        results.append({**run_single_llm(ep), "country": ep.country})
        results.append({**run_multi_agent_simple(ep), "country": ep.country})
        results.append({**run_multi_agent_llm_coord(ep), "country": ep.country})
        results.append({**run_ablation_no_veto(ep), "country": ep.country})
        results.append({**run_ablation_no_comm(ep), "country": ep.country})

    df_res = pd.DataFrame(results)

    cols = ["score", "forecast_mape", "cost", "emission", "security", "violations", "messages", "rounds", "vetos", "runtime_sec"]
    benchmark_mean = df_res.groupby("method")[cols].mean().sort_values("score", ascending=False)

    print("\n" + "="*60)
    print("BENCHMARK SONUCU (FAIR + High Score + Soft Constraints) - MEAN")
    print("="*60)
    print(benchmark_mean)

    benchmark_std = df_res.groupby("method")[cols].std().fillna(0.0)

    benchmark_mean.to_csv("benchmark_last_results_fair_highscore_mean.csv")
    benchmark_std.to_csv("benchmark_last_results_fair_highscore_std.csv")
    df_res.to_csv("benchmark_last_results_fair_highscore_raw.csv", index=False)

else:
    print("Veri hatası.")


Seçilen Ülke Sayısı: 15
Simülasyon başlıyor (FAIR + High Score + Soft Constraints + Candidate Search)...
[1/15] İşleniyor: Austria ...
[2/15] İşleniyor: Belgium ...
[3/15] İşleniyor: Bulgaria ...
[4/15] İşleniyor: Croatia ...
[5/15] İşleniyor: Cyprus ...
[6/15] İşleniyor: Czechia ...
[7/15] İşleniyor: Denmark ...
[8/15] İşleniyor: Estonia ...
[9/15] İşleniyor: Finland ...
[10/15] İşleniyor: France ...
[11/15] İşleniyor: Germany ...
[12/15] İşleniyor: Greece ...
[13/15] İşleniyor: Hungary ...
[14/15] İşleniyor: Ireland ...
[15/15] İşleniyor: Italy ...

BENCHMARK SONUCU (FAIR + High Score + Soft Constraints) - MEAN
                            score  forecast_mape      cost  emission  \
method                                                                 
multi_agent_simple_agg   0.825129       4.377175  0.066667  0.325234   
multi_agent_llm_coord    0.814956       4.377175  0.112994  0.307037   
ablation_no_comm         0.814954       4.377175  0.112333  0.307340   
ablation_no_veto   